<h1 style='color:black'> Context Manager </h1>

The main benefit of using context manager is to be on top of handling resources; see https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/. Besides files, `with` is also useful in dealing with threads, DBs, sockets, etc. 

In [1]:
# from __future__ import with_statement
## This isn't required from Python 2.7 onwards.
# 
#with open('hello.txt') as f: # f is the handle and it is optional
#    for line in f:
#        print line

class Ctx_Mgr(object):
    def __enter__(self):
        print('enter')
        return 42
    
    def __exit__(self, exc_type, exc_obj, exc_tb): # The last three variables are for exceptions handling - if there is no exceptions, exc variables will be None.
        if exc_type is not None:
            print('exception')
            return True # This is the line needed for exceptions to be truly handled - by default, this exit function does not return True
        print('exit')
        
with Ctx_Mgr as val:
    # The protocol of the context manager...
    # (1) Obj = Ctx_Mgr(): similar in open(file), which returns an object
    # (2) obj.__enter__() and is given to the variable after 'as' (if any) and it is a global variable!
    # (3) execute the body, as below
    
    # 10 / 0 if there is an exception in the body, control is haneded over to exit directly and all three exc variables will be non-trivially populated.
    print(42)
    print('in body')
    
    # (4) obj.__exit__()
    


AttributeError: __exit__

Compare below for class solution vs. decorator solution to context manager to auto save.

In [ ]:
"""Class solution"""

import pickle

class AutoSave(object):
    def __init__(self, d):
        self.d = d
    def __enter__(self):
        print('enter')
        self.fo = open('data.out', 'w')
        return self.d
    def __exit__(self, exc_type, exc_obj, exc_tb):
        if exc_type is not None:
            print('Exception!')
            return True
        
        pickle.dump(self.d, self.fo)
        self.fo.close()
        print('exit')
        
d = {}
with AutoSave(d) as h:
    d['usa'] = 1
    d['uk'] = 44
    # 10/ 0
    
for key, value in h.items(): #Again h is a global variable!
    print("{}:{}".format(key, value))
    
"""Function/decorator solution"""

# But to understand this solution you need to understand the class solution first.

import pickle
from contextlib import contextmanager

@contextmanager
def autosave(d):
    # No __init__ method needed
    
    # First the __enter__ part
    print('enter')
    fo = open('data.out', 'w')
    
    try:
        yield # yielding control to the decorator
    except Exception:
        print('Exception!')
        
    # The the __exit__ part
    pickle.dump(d, fo)
    fo.close()
    print('exit')
    
    